<a href="https://colab.research.google.com/github/JasonAlva/practice/blob/main/Fake_News_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import  re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
data=pd.read_csv('/content/dataset.csv',on_bad_lines='skip',engine='python')

In [8]:
data.head()

,title,text,unreliable,author
0,WARNING: A Pivotal Moment For The Stock Market...,WARNING: A Pivotal Moment For The Stock Market...,1,Anonymous Coward (UID 72071746)
1,"Trump, top defense officials, discuss North Ko...",WASHINGTON - U.S. President Donald Trump met ...,0,Unknown
2,British civil servants' union calls nationwide...,LONDON - British civil servants will vote nex...,0,Unknown
3,A**hole Of The Day – Michele Bachmann: Muslim...,Michele Bachmann has been pretty quiet since l...,1,Unknown
4,’Deport Fat People’ Posters Appear At CU Bould...,Posters calling on Donald Trump to “Deport Fat...,0,Lucas Nolan


In [9]:
data.tail()

,title,text,unreliable,author
8195,CRAZY VIDEO OF MUSLIM JIHADESS Being Dragged U...,Feel good video of the day A female ISIS recru...,1,Unknown
8196,Wikileaks: Bill Clinton BOASTS of Hillary’s ‘W...,Print \nThe bombshells about this criminal are...,1,Pamela Geller
8197,LAURA INGRAHAM: GOP Senators In Private Meetin...,During a private meeting of the most senior Re...,1,Unknown
8198,Sometimes the road to diet change can be fun,Sometimes the road to diet change can be fun A...,1,Hesh Goldstein
8199,Lynch says meeting with Bill Clinton 'casts a ...,WASHINGTON - U.S. Attorney General Loretta Ly...,0,Unknown


In [10]:
data.isnull().sum()

,0
title,69
text,4
unreliable,0
author,235


In [11]:
data=data.fillna('')

In [12]:
data['content']=data['author']+' '+data['title']

In [13]:
data.head()

,title,text,unreliable,author,content
0,WARNING: A Pivotal Moment For The Stock Market...,WARNING: A Pivotal Moment For The Stock Market...,1,Anonymous Coward (UID 72071746),Anonymous Coward (UID 72071746) WARNING: A Piv...
1,"Trump, top defense officials, discuss North Ko...",WASHINGTON - U.S. President Donald Trump met ...,0,Unknown,"Unknown Trump, top defense officials, discuss ..."
2,British civil servants' union calls nationwide...,LONDON - British civil servants will vote nex...,0,Unknown,Unknown British civil servants' union calls na...
3,A**hole Of The Day – Michele Bachmann: Muslim...,Michele Bachmann has been pretty quiet since l...,1,Unknown,Unknown A**hole Of The Day – Michele Bachmann...
4,’Deport Fat People’ Posters Appear At CU Bould...,Posters calling on Donald Trump to “Deport Fat...,0,Lucas Nolan,Lucas Nolan ’Deport Fat People’ Posters Appear...


In [14]:
X=data.drop(columns='unreliable',axis=1)
Y=data['unreliable']

In [15]:
port_stem=PorterStemmer()

#stemming

In [16]:
def stemming(content):
    stem_word=re.sub('[^a-zA-z]',' ',content)
    stem_word=stem_word.lower()
    stem_word=stem_word.split()
    stem_word=[port_stem.stem(word) for word in stem_word if not word in stopwords.words('english')]
    stem_word=' '.join(stem_word)
    return stem_word

In [17]:
data['content']=data['content'].apply(stemming)

In [18]:
print(data['content'])

0       anonym coward uid warn pivot moment stock mark...
1       unknown trump top defens offici discuss north ...
2       unknown british civil servant union call natio...
3       unknown hole day michel bachmann muslim rape j...
4       luca nolan deport fat peopl poster appear cu b...
                              ...                        
8195    unknown crazi video muslim jihadess drag hill ...
8196    pamela geller wikileak bill clinton boast hill...
8197    unknown laura ingraham gop senat privat meet l...
8198           hesh goldstein sometim road diet chang fun
8199    unknown lynch say meet bill clinton cast shado...
Name: content, Length: 8200, dtype: object


In [19]:
#seperating values and labels
X=data['content'].values
Y=data['unreliable'].values
valid_mask = Y != ''
X = X[valid_mask]
Y = Y[valid_mask]


In [20]:
print(X)

['anonym coward uid warn pivot moment stock market may'
 'unknown trump top defens offici discuss north korea option white hous'
 'unknown british civil servant union call nationwid strike ballot' ...
 'unknown laura ingraham gop senat privat meet laugh loud idea trump border wall would ever built'
 'hesh goldstein sometim road diet chang fun'
 'unknown lynch say meet bill clinton cast shadow email probe']


In [21]:
print(Y)

[1 0 0 ... 1 1 0]


In [22]:
#using tfid vectorizer
vec=TfidfVectorizer()
vec.fit(X)
X=vec.transform(X)

In [23]:
mask = Y != ''

# Apply the mask to filter out empty string classes
X = X[mask]
Y = Y[mask]


In [24]:
X_Train,X_Test,Y_Train,Y_Test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [26]:
print(X_Train.shape,X_Test.shape)


(6560, 9516) (1640, 9516)
(8200, 9516)


In [27]:
#training the model
model=LogisticRegression()
model.fit(X_Train,Y_Train)

LogisticRegression()

In [28]:
#model evaluation
Training_prediction=model.predict(X_Train)
Training_prediction_accuracy=accuracy_score(Training_prediction,Y_Train)
print("training score:",Training_prediction_accuracy)

training score: 0.963719512195122


In [29]:
Testing_prediction=model.predict(X_Test)
Testing_prediction_accuracy=accuracy_score(Testing_prediction,Y_Test)
print("Testing score:",Testing_prediction_accuracy)

Testing score: 0.8987804878048781


In [38]:
value=model.predict(X_Test[11])
print(value)
print(Y_Test[11])
if value==Y_Test[11] :
  print("correct prediction")
else:
  print("wrong prediction")

[0]
0
correct prediction
